### Librerias útiles

In [1]:
# libreria que utilizamos para manejar la API de twitter
require 'twitter'
# libreria para el manejo de dataframe
require 'rover-df'
require 'vega'

true

In [2]:
# definicion del token de autenticacion de la API
client = Twitter::REST::Client.new do |config|
    config.consumer_key        = ""
    config.consumer_secret     = ""
    config.access_token        = ""
    config.access_token_secret = ""
end

#<Twitter::REST::Client:0x0000018ed6bd89e8 @consumer_key="H2cC7eKrsCHqBoBZG6XYNeFAi", @consumer_secret="z2d9Riqn3JyeOvkQ0R0E6P89Huo22SEB92GdwAbK69SsS4zt3i", @access_token="1443289288817483787-Y23CyxwSuGhXX0M6yZzGKbmsvqNMQV", @access_token_secret="18nVpC79FpzY46PrkAQiB9gbXCT7XTRvoL1bJQQbtXIDr">

### Recuperamos los tweets para el conjunto 1

In [55]:
# se crea el dataframe que contendra cada uno de los tweets
allTweets = Rover::DataFrame.new()
# Crea o abre un nuevo fichero, y escribe
File.open('conjunto_1.txt', 'w') do |fichero|
  # recuperacion de los tweets, sin considerar los RT's
  client.search("a OR e OR i OR o OR u since:2021-11-1 until:2021-11-7",
                result_type: "recent",
                lang: "es",
                tweet_mode: "extended").take(500).each do |tweet|
    if (tweet.text[0,2]!='RT')

      # guardamos el contenido del tweet, que es lo que nos interesa mas adelante
      fichero.puts(tweet.text)
      # creamos un dataframe auxiliar
      df = Rover::DataFrame.new([
      {id_tweet: tweet.id, name: tweet.user.screen_name, date: tweet.created_at.strftime("%Y-%m-%d"), text: tweet.text}
      ])
      # para concatenarlo al original
      allTweets.concat(df)
    end
  end
end

puts "ok"

ok


### Recuperamos los tweets para el conjunto 2

In [56]:
# se crea el dataframe que contendra cada uno de los tweets
tweets = Rover::DataFrame.new()
# Crea o abre un nuevo fichero, y escribe
File.open('conjunto_2.txt', 'w') do |fichero|
  # recuperacion de los tweets, sin considerar los RT's
  client.search("Berizzo since:2021-11-1 until:2021-11-7",
                result_type: "recent",
                lang: "es",
                tweet_mode: "extended").take(500).each do |tweet|
    if (tweet.text[0,2]!='RT')

      # guardamos el contenido del tweet, que es lo que nos interesa mas adelante
      fichero.puts(tweet.text)
      # creamos un dataframe auxiliar
      df = Rover::DataFrame.new([
      {id_tweet: tweet.id, name: tweet.user.screen_name, date: tweet.created_at.strftime("%Y-%m-%d"), text: tweet.text}
      ])
      # para concatenarlo al original
      tweets.concat(df)
    end
  end
end

puts "ok"

ok


### Procedemos a limpiar el texto extraido

#### Limpieza del fichero 1

In [57]:
File.open('conjunto_1_limpio.txt', 'w') do |fichero|
  # con esto leemos el contenido del archivo conjunto_1.txt por lineas
  lineas = File.readlines("conjunto_1.txt")
  lineas.each do |linea|
    # limpiamos los arroba
    linea_limpia = linea.gsub(/@\w+/, "")
    # limpiamos los links
    linea_limpia = linea_limpia.gsub(/http\S+/, "")
    # limpiamos los hashtag
    linea_limpia = linea_limpia.gsub(/#\S+/, "")
    # sirve para eliminar cualquier caracter que no se encuentre listado en []
    # como numeros, emojis, caracteres especiales, etc
    linea_limpia = linea_limpia.gsub(/[^A-Za-záéíóú\s]/, "")
    # cambiamos los acentos por la letra sin acento
    linea_limpia = linea_limpia.gsub(/á/, "a")
    linea_limpia = linea_limpia.gsub(/é/, "e")
    linea_limpia = linea_limpia.gsub(/í/, "i")
    linea_limpia = linea_limpia.gsub(/ó/, "o")
    linea_limpia = linea_limpia.gsub(/ú/, "u")
    linea_limpia = linea_limpia.gsub(/Á+/, "A")
    linea_limpia = linea_limpia.gsub(/É+/, "E")
    linea_limpia = linea_limpia.gsub(/Í+/, "i")
    linea_limpia = linea_limpia.gsub(/Ó+/, "o")
    linea_limpia = linea_limpia.gsub(/Ú+/, "u")
    fichero.puts(linea_limpia)
  end
end

puts "ok"

ok


#### Limpieza del fichero 2

In [58]:
File.open('conjunto_2_limpio.txt', 'w') do |fichero|
  # con esto leemos el contenido del archivo conjunto_1.txt por lineas
  lineas = File.readlines("conjunto_2.txt")
  lineas.each do |linea|
    # limpiamos los arroba
    linea_limpia = linea.gsub(/@\w+/, "")
    # limpiamos los links
    linea_limpia = linea_limpia.gsub(/http\S+/, "")
    # limpiamos los hashtag
    linea_limpia = linea_limpia.gsub(/#\S+/, "")
    # sirve para eliminar cualquier caracter que no se encuentre listado en []
    # como numeros, emojis, caracteres especiales, etc
    linea_limpia = linea_limpia.gsub(/[^A-Za-záéíóú\s]/, "")
    # cambiamos los acentos por la letra sin acento
    linea_limpia = linea_limpia.gsub(/á/, "a")
    linea_limpia = linea_limpia.gsub(/é/, "e")
    linea_limpia = linea_limpia.gsub(/í/, "i")
    linea_limpia = linea_limpia.gsub(/ó/, "o")
    linea_limpia = linea_limpia.gsub(/ú/, "u")
    linea_limpia = linea_limpia.gsub(/Á+/, "A")
    linea_limpia = linea_limpia.gsub(/É+/, "E")
    linea_limpia = linea_limpia.gsub(/Í+/, "i")
    linea_limpia = linea_limpia.gsub(/Ó+/, "o")
    linea_limpia = linea_limpia.gsub(/Ú+/, "u")
    fichero.puts(linea_limpia)
  end
end

puts "ok"

ok


#### Escribimos las palabras en un DataFrame a excepcion de las no deseadas o cortas

##### Conteo de palabras del conjunto 1

In [59]:
# creamos el dataframe que contendra todas las palabras encontradas en el conjunto 1
conjunto_1 = Rover::DataFrame.new()
# vamos leyendo el archivo por lineas
lineas = File.readlines("conjunto_1_limpio.txt")
lineas.each do |linea|
  # divide el contenido por espacios para encontrar las palabras
  palabras = linea.split(" ")
  palabras.each do |palabra|
    # restringimos las palabras cortas y aquellas que no deseamos analisar
    if (palabra.length > 3)
      # dataframe auxiliar Palabras es el nombre de la columna y palabra será lo asignado
      df = Rover::DataFrame.new([{Palabras: palabra.downcase}])
      # concatenamos al DataFrame original
      conjunto_1.concat(df)
    end
  end
end

puts conjunto_1.count

# conjunto_1[:Palabras].tally

2122


##### Conteo de palabras del conjunto 2

In [60]:
# creamos el dataframe que contendra todas las palabras encontradas en el conjunto 2
conjunto_2 = Rover::DataFrame.new()
# vamos leyendo el archivo por lineas
lineas = File.readlines("conjunto_2_limpio.txt")
lineas.each do |linea|
  # divide el contenido por espacios para encontrar las palabras
  palabras = linea.split(" ")
  palabras.each do |palabra|
    # restringimos las palabras cortas y aquellas que no deseamos analisar
    if (palabra.length > 3)
      # dataframe auxiliar Palabras es el nombre de la columna y palabra será lo asignado
      df = Rover::DataFrame.new([{Palabras: palabra.downcase}])
      # concatenamos al DataFrame original
      conjunto_2.concat(df)
    end
  end
end

puts conjunto_2.count

# conjunto_1[:Palabras].tally

2922


### Obtenemos el objeto a graficar

#### Conjunto 1

In [61]:
lista_1 = []
conjunto_1[:Palabras].tally.each do |llave, valor|
  lista_1.push({Palabras: llave, Frecuencia: valor})
end

puts lista_1.length

1472


#### Grafico del item 1

In [62]:
Vega.lite
  .title("Grafico 1 - Histograma de palabras del conjunto 1")
  .data(lista_1)
  .transform([
    {
      "window": [{
        "op": "rank",
        "as": "rank"
      }],
      "sort": [{ "field": "Frecuencia", "order": "descending" }]
    }, {
      "filter": "datum.rank <= 20"
    }
  ])
  .mark(type: "bar", tooltip: true)
  .encoding(
    x: {field: "Palabras", type: "nominal", sort: "-y"},
    y: {field: "Frecuencia", type: "quantitative"}
  )

#<Vega::LiteChart:0x0000018ed5e64c30 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "title"=>"Grafico 1 - Histograma de palabras del conjunto 1", "data"=>{:values=>[{:Palabras=>"abono", :Frecuencia=>1}, {:Palabras=>"para", :Frecuencia=>29}, {:Palabras=>"decia", :Frecuencia=>1}, {:Palabras=>"despues", :Frecuencia=>3}, {:Palabras=>"preguntan", :Frecuencia=>1}, {:Palabras=>"porque", :Frecuencia=>8}, {:Palabras=>"pocos", :Frecuencia=>1}, {:Palabras=>"pacientes", :Frecuencia=>2}, {:Palabras=>"tienen", :Frecuencia=>3}, {:Palabras=>"ojeriza", :Frecuencia=>1}, {:Palabras=>"medicos", :Frecuencia=>3}, {:Palabras=>"trabajan", :Frecuencia=>1}, {:Palabras=>"hospitales", :Frecuencia=>1}, {:Palabras=>"publicos", :Frecuencia=>1}, {:Palabras=>"todo", :Frecuencia=>12}, {:Palabras=>"enfermeras", :Frecuencia=>1}, {:Palabras=>"empaticos", :Frecuencia=>1}, {:Palabras=>"humanos", :Frecuencia=>3}, {:Palabras=>"merecen", :Frecuencia=>1}, {:Palabras=>"llamarse", :Frecuencia=>1}, {:Palabras=>"pero", :Frecuencia=>23}, {:Palabras=>"igual", :Frecuencia=>9}, {:Palabras=>"comia", :Frecuencia=>1}, {:Palabras=>"jugar", :Frecuencia=>3}, {:Palabras=>"pwro", :Frecuencia=>1}, {:Palabras=>"pareces", :Frecuencia=>1}, {:Palabras=>"patetica", :Frecuencia=>1}, {:Palabras=>"inutil", :Frecuencia=>1}, {:Palabras=>"osea", :Frecuencia=>1}, {:Palabras=>"hubieras", :Frecuencia=>1}, {:Palabras=>"visto", :Frecuencia=>3}, {:Palabras=>"siempre", :Frecuencia=>5}, {:Palabras=>"visitendo", :Frecuencia=>1}, {:Palabras=>"negro", :Frecuencia=>1}, {:Palabras=>"estando", :Frecuencia=>1}, {:Palabras=>"encerrado", :Frecuencia=>1}, {:Palabras=>"cuarto", :Frecuencia=>1}, {:Palabras=>"cuando", :Frecuencia=>16}, {:Palabras=>"descompuso", :Frecuencia=>1}, {:Palabras=>"escapar", :Frecuencia=>1}, {:Palabras=>"clases", :Frecuencia=>1}, {:Palabras=>"evitar", :Frecuencia=>1}, {:Palabras=>"bullyado", :Frecuencia=>1}, {:Palabras=>"life", :Frecuencia=>2}, {:Palabras=>"cada", :Frecuencia=>7}, {:Palabras=>"ando", :Frecuencia=>2}, {:Palabras=>"flex", :Frecuencia=>1}, {:Palabras=>"engordando", :Frecuencia=>1}, {:Palabras=>"racks", :Frecuencia=>2}, {:Palabras=>"bitch", :Frecuencia=>1}, {:Palabras=>"italia", :Frecuencia=>1}, {:Palabras=>"marca", :Frecuencia=>1}, {:Palabras=>"intentos", :Frecuencia=>1}, {:Palabras=>"increible", :Frecuencia=>2}, {:Palabras=>"cantidad", :Frecuencia=>2}, {:Palabras=>"oportunidades", :Frecuencia=>1}, {:Palabras=>"desperdicia", :Frecuencia=>1}, {:Palabras=>"volumen", :Frecuencia=>1}, {:Palabras=>"generacion", :Frecuencia=>1}, {:Palabras=>"juego", :Frecuencia=>1}, {:Palabras=>"necesitamos", :Frecuencia=>2}, {:Palabras=>"killer", :Frecuencia=>1}, {:Palabras=>"francotirador", :Frecuencia=>1}, {:Palabras=>"tipo", :Frecuencia=>1}, {:Palabras=>"escopeta", :Frecuencia=>1}, {:Palabras=>"rubio", :Frecuencia=>1}, {:Palabras=>"morocho", :Frecuencia=>1}, {:Palabras=>"zuluaga", :Frecuencia=>1}, {:Palabras=>"hijueputa", :Frecuencia=>1}, {:Palabras=>"depende", :Frecuencia=>1}, {:Palabras=>"salis", :Frecuencia=>1}, {:Palabras=>"barsito", :Frecuencia=>1}, {:Palabras=>"jean", :Frecuencia=>1}, {:Palabras=>"venis", :Frecuencia=>1}, {:Palabras=>"casa", :Frecuencia=>3}, {:Palabras=>"pongas", :Frecuencia=>2}, {:Palabras=>"nada", :Frecuencia=>14}, {:Palabras=>"esta", :Frecuencia=>14}, {:Palabras=>"noche", :Frecuencia=>5}, {:Palabras=>"saldre", :Frecuencia=>1}, {:Palabras=>"janguear", :Frecuencia=>1}, {:Palabras=>"tengo", :Frecuencia=>3}, {:Palabras=>"miedo", :Frecuencia=>6}, {:Palabras=>"verdad", :Frecuencia=>3}, {:Palabras=>"gusta", :Frecuencia=>1}, {:Palabras=>"mucho", :Frecuencia=>5}, {:Palabras=>"regalen", :Frecuencia=>1}, {:Palabras=>"flores", :Frecuencia=>1}, {:Palabras=>"mama", :Frecuencia=>4}, {:Palabras=>"descubra", :Frecuencia=>1}, {:Palabras=>"recien", :Frecuencia=>1}, {:Palabras=>"empezar", :Frecuencia=>2}, {:Palabras=>"asustar", :Frecuencia=>1}, {:Palabras=>"anda", :F

#### Conjunto 2

In [63]:
lista_2 = []
conjunto_2[:Palabras].tally.each do |llave, valor|
  lista_2.push({Palabras: llave, Frecuencia: valor})
end

puts lista_2.length

1414


#### Grafico del item 2

In [64]:
Vega.lite
  .title("Grafico 2 - Histograma de palabras del conjunto 2")
  .data(lista_2)
  .transform([
      {
        "window": [{
          "op": "rank",
          "as": "rank"
        }],
        "sort": [{ "field": "Frecuencia", "order": "descending" }]
      }, {
        "filter": "datum.rank <= 20"
      }
    ])
  .mark(type: "bar", tooltip: true)
  .encoding(
    x: {field: "Palabras", type: "nominal", sort: "-y"},
    y: {field: "Frecuencia", type: "quantitative"}
  )



#<Vega::LiteChart:0x0000018ed6a113f8 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "title"=>"Grafico 2 - Histograma de palabras del conjunto 2", "data"=>{:values=>[{:Palabras=>"pasar", :Frecuencia=>2}, {:Palabras=>"comision", :Frecuencia=>1}, {:Palabras=>"berizzo", :Frecuencia=>207}, {:Palabras=>"harrison", :Frecuencia=>3}, {:Palabras=>"cuando", :Frecuencia=>10}, {:Palabras=>"tienes", :Frecuencia=>2}, {:Palabras=>"buen", :Frecuencia=>4}, {:Palabras=>"entrenador", :Frecuencia=>13}, {:Palabras=>"buena", :Frecuencia=>2}, {:Palabras=>"idea", :Frecuencia=>4}, {:Palabras=>"podrias", :Frecuencia=>1}, {:Palabras=>"permitirte", :Frecuencia=>1}, {:Palabras=>"lujo", :Frecuencia=>2}, {:Palabras=>"aguantar", :Frecuencia=>1}, {:Palabras=>"hasta", :Frecuencia=>12}, {:Palabras=>"final", :Frecuencia=>3}, {:Palabras=>"cesarlo", :Frecuencia=>1}, {:Palabras=>"cuantos", :Frecuencia=>1}, {:Palabras=>"entrenadores", :Frecuencia=>6}, {:Palabras=>"estuvieron", :Frecuencia=>1}, {:Palabras=>"aguantaron", :Frecuencia=>2}, {:Palabras=>"acabaron", :Frecuencia=>1}, {:Palabras=>"triunfando", :Frecuencia=>1}, {:Palabras=>"ejemplo", :Frecuencia=>1}, {:Palabras=>"celta", :Frecuencia=>6}, {:Palabras=>"lejos", :Frecuencia=>1}, {:Palabras=>"rechazo", :Frecuencia=>2}, {:Palabras=>"renuncio", :Frecuencia=>1}, {:Palabras=>"terminos", :Frecuencia=>1}, {:Palabras=>"distintos", :Frecuencia=>1}, {:Palabras=>"juan", :Frecuencia=>3}, {:Palabras=>"escobar", :Frecuencia=>3}, {:Palabras=>"dijo", :Frecuencia=>7}, {:Palabras=>"jugar", :Frecuencia=>8}, {:Palabras=>"seleccion", :Frecuencia=>15}, {:Palabras=>"igual", :Frecuencia=>7}, {:Palabras=>"melgarejo", :Frecuencia=>1}, {:Palabras=>"partido", :Frecuencia=>3}, {:Palabras=>"echaron", :Frecuencia=>2}, {:Palabras=>"bueno", :Frecuencia=>4}, {:Palabras=>"comparandolo", :Frecuencia=>1}, {:Palabras=>"hacer", :Frecuencia=>2}, {:Palabras=>"semanitas", :Frecuencia=>1}, {:Palabras=>"gracias", :Frecuencia=>1}, {:Palabras=>"athletic", :Frecuencia=>3}, {:Palabras=>"gane", :Frecuencia=>1}, {:Palabras=>"pierda", :Frecuencia=>2}, {:Palabras=>"sigo", :Frecuencia=>2}, {:Palabras=>"diciendo", :Frecuencia=>2}, {:Palabras=>"mismo", :Frecuencia=>9}, {:Palabras=>"ziganda", :Frecuencia=>4}, {:Palabras=>"garitano", :Frecuencia=>4}, {:Palabras=>"nada", :Frecuencia=>10}, {:Palabras=>"cambia", :Frecuencia=>2}, {:Palabras=>"porque", :Frecuencia=>4}, {:Palabras=>"seguimos", :Frecuencia=>1}, {:Palabras=>"doble", :Frecuencia=>1}, {:Palabras=>"pivote", :Frecuencia=>1}, {:Palabras=>"sirve", :Frecuencia=>2}, {:Palabras=>"para", :Frecuencia=>29}, {:Palabras=>"seguiremos", :Frecuencia=>1}, {:Palabras=>"cambie", :Frecuencia=>1}, {:Palabras=>"todo", :Frecuencia=>12}, {:Palabras=>"contratar", :Frecuencia=>1}, {:Palabras=>"asco", :Frecuencia=>3}, {:Palabras=>"seor", :Frecuencia=>2}, {:Palabras=>"jugadores", :Frecuencia=>18}, {:Palabras=>"seleccin", :Frecuencia=>2}, {:Palabras=>"puede", :Frecuencia=>9}, {:Palabras=>"probar", :Frecuencia=>2}, {:Palabras=>"estos", :Frecuencia=>13}, {:Palabras=>"momentos", :Frecuencia=>2}, {:Palabras=>"solo", :Frecuencia=>11}, {:Palabras=>"pequea", :Frecuencia=>2}, {:Palabras=>"parte", :Frecuencia=>4}, {:Palabras=>"problema", :Frecuencia=>5}, {:Palabras=>"basta", :Frecuencia=>2}, {:Palabras=>"dimision", :Frecuencia=>1}, {:Palabras=>"vuelva", :Frecuencia=>2}, {:Palabras=>"acaba", :Frecuencia=>1}, {:Palabras=>"segunda", :Frecuencia=>2}, {:Palabras=>"chachoneta", :Frecuencia=>2}, {:Palabras=>"peor", :Frecuencia=>2}, {:Palabras=>"vesga", :Frecuencia=>2}, {:Palabras=>"puta", :Frecuencia=>1}, {:Palabras=>"mierda", :Frecuencia=>3}, {:Palabras=>"valverde", :Frecuencia=>2}, {:Palabras=>"hizo", :Frecuencia=>5}, {:Palabras=>"debutar", :Frecuencia=>1}, {:Palabras=>"gijon", :Frecuencia=>1}, {:Palabras=>"dani", :Frecuencia=>2}, {:Palabras=>"garcia", :Frecuencia=>1}, {:Palabras=>"puso", :Frecuencia=>2}, {:Palabras=>"tambien", :F

### Intersección de los conjuntos 1 y 2

In [65]:
df_interseccion = Rover::DataFrame.new()
df_interseccion = conjunto_1.inner_join(conjunto_2, on: :Palabras)
puts df_interseccion.count

4632


#### Lista de palabras en la Intersección

In [66]:
lista_interseccion = []
df_interseccion[:Palabras].tally.each do |llave, valor|
  lista_interseccion.push({Palabras: llave, Frecuencia: valor})
end

puts "ok"

ok


#### Gráfico de la intersección

In [87]:
Vega.lite
  .title("Grafico 3 - Histograma de palabras en la Intersección")
  .data(lista_interseccion)
  .transform([
      {
        "window": [{
          "op": "rank",
          "as": "rank"
        }],
        "sort": [{ "field": "Frecuencia", "order": "descending" }]
      }, {
        "filter": "datum.rank <= 20"
      }
    ])
  .mark(type: "bar", tooltip: true)
  .encoding(
    x: {field: "Palabras", type: "nominal", sort: "-y"},
    y: {field: "Frecuencia", type: "quantitative"}
  )

#<Vega::LiteChart:0x0000018ed8a24960 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "title"=>"Grafico 3 - Histograma de palabras en la Intersección", "data"=>{:values=>[{:Palabras=>"para", :Frecuencia=>841}, {:Palabras=>"despues", :Frecuencia=>12}, {:Palabras=>"porque", :Frecuencia=>32}, {:Palabras=>"pocos", :Frecuencia=>1}, {:Palabras=>"tienen", :Frecuencia=>6}, {:Palabras=>"todo", :Frecuencia=>144}, {:Palabras=>"merecen", :Frecuencia=>1}, {:Palabras=>"pero", :Frecuencia=>529}, {:Palabras=>"igual", :Frecuencia=>63}, {:Palabras=>"jugar", :Frecuencia=>24}, {:Palabras=>"visto", :Frecuencia=>9}, {:Palabras=>"siempre", :Frecuencia=>25}, {:Palabras=>"cuando", :Frecuencia=>160}, {:Palabras=>"cada", :Frecuencia=>21}, {:Palabras=>"marca", :Frecuencia=>1}, {:Palabras=>"juego", :Frecuencia=>2}, {:Palabras=>"necesitamos", :Frecuencia=>2}, {:Palabras=>"pongas", :Frecuencia=>2}, {:Palabras=>"nada", :Frecuencia=>140}, {:Palabras=>"esta", :Frecuencia=>294}, {:Palabras=>"tengo", :Frecuencia=>9}, {:Palabras=>"verdad", :Frecuencia=>3}, {:Palabras=>"gusta", :Frecuencia=>6}, {:Palabras=>"mucho", :Frecuencia=>20}, {:Palabras=>"recien", :Frecuencia=>2}, {:Palabras=>"anda", :Frecuencia=>1}, {:Palabras=>"lombardi", :Frecuencia=>4}, {:Palabras=>"pizza", :Frecuencia=>1}, {:Palabras=>"ustedes", :Frecuencia=>6}, {:Palabras=>"bien", :Frecuencia=>18}, {:Palabras=>"pais", :Frecuencia=>14}, {:Palabras=>"estoy", :Frecuencia=>12}, {:Palabras=>"pueden", :Frecuencia=>6}, {:Palabras=>"pronto", :Frecuencia=>2}, {:Palabras=>"pueblo", :Frecuencia=>3}, {:Palabras=>"quien", :Frecuencia=>40}, {:Palabras=>"tras", :Frecuencia=>2}, {:Palabras=>"contra", :Frecuencia=>6}, {:Palabras=>"vacuna", :Frecuencia=>2}, {:Palabras=>"edad", :Frecuencia=>2}, {:Palabras=>"antes", :Frecuencia=>24}, {:Palabras=>"tanto", :Frecuencia=>6}, {:Palabras=>"tiene", :Frecuencia=>78}, {:Palabras=>"bueno", :Frecuencia=>24}, {:Palabras=>"carlos", :Frecuencia=>6}, {:Palabras=>"tampoco", :Frecuencia=>6}, {:Palabras=>"hacer", :Frecuencia=>22}, {:Palabras=>"nosotros", :Frecuencia=>2}, {:Palabras=>"titulo", :Frecuencia=>1}, {:Palabras=>"solo", :Frecuencia=>66}, {:Palabras=>"trabajo", :Frecuencia=>3}, {:Palabras=>"sobre", :Frecuencia=>3}, {:Palabras=>"lunes", :Frecuencia=>2}, {:Palabras=>"semana", :Frecuencia=>1}, {:Palabras=>"desde", :Frecuencia=>15}, {:Palabras=>"sanchez", :Frecuencia=>1}, {:Palabras=>"esos", :Frecuencia=>6}, {:Palabras=>"entonces", :Frecuencia=>2}, {:Palabras=>"tienes", :Frecuencia=>4}, {:Palabras=>"dijo", :Frecuencia=>7}, {:Palabras=>"seguramente", :Frecuencia=>1}, {:Palabras=>"tiempo", :Frecuencia=>9}, {:Palabras=>"ganador", :Frecuencia=>1}, {:Palabras=>"orden", :Frecuencia=>2}, {:Palabras=>"otros", :Frecuencia=>4}, {:Palabras=>"hasta", :Frecuencia=>84}, {:Palabras=>"estos", :Frecuencia=>117}, {:Palabras=>"saben", :Frecuencia=>8}, {:Palabras=>"sino", :Frecuencia=>1}, {:Palabras=>"unos", :Frecuencia=>4}, {:Palabras=>"esto", :Frecuencia=>4}, {:Palabras=>"espaa", :Frecuencia=>4}, {:Palabras=>"motivo", :Frecuencia=>1}, {:Palabras=>"haya", :Frecuencia=>2}, {:Palabras=>"cambio", :Frecuencia=>6}, {:Palabras=>"ahora", :Frecuencia=>77}, {:Palabras=>"mismo", :Frecuencia=>45}, {:Palabras=>"presidente", :Frecuencia=>2}, {:Palabras=>"maana", :Frecuencia=>6}, {:Palabras=>"seria", :Frecuencia=>6}, {:Palabras=>"tambien", :Frecuencia=>56}, {:Palabras=>"dejar", :Frecuencia=>2}, {:Palabras=>"hacerlo", :Frecuencia=>2}, {:Palabras=>"epoca", :Frecuencia=>2}, {:Palabras=>"mayor", :Frecuencia=>3}, {:Palabras=>"dios", :Frecuencia=>8}, {:Palabras=>"virgen", :Frecuencia=>1}, {:Palabras=>"jorge", :Frecuencia=>1}, {:Palabras=>"quiero", :Frecuencia=>5}, {:Palabras=>"todos", :Frecuencia=>36}, {:Palabras=>"hacen", :Frecuencia=>2}, {:Palabras=>"linea", :Frecuencia=>1}, {:Palabras=>"entre", :Frecuencia=>1}, {:Palabras=>"vida", :Frecuencia=>18}, {:Palabras=>"cosas", :Frecuencia=>8}, {:Palabras=>

### Union de los conjuntos 1 y 2

In [70]:
df_union = Rover::DataFrame.new()
df_union = conjunto_1.concat(conjunto_2)
puts "ok"

ok


#### Lista de palabras en la unión

In [71]:
lista_union = []
df_union[:Palabras].tally.each do |llave, valor|
  lista_union.push({Palabras: llave, Frecuencia: valor})
end

puts "ok"

ok


#### Gráfico de la unión

In [72]:
Vega.lite
  .title("Grafico 4 - Histograma de palabras en la Unión")
  .data(lista_union)
  .transform([
      {
        "window": [{
          "op": "rank",
          "as": "rank"
        }],
        "sort": [{ "field": "Frecuencia", "order": "descending" }]
      }, {
        "filter": "datum.rank <= 20"
      }
    ])
  .mark(type: "bar", tooltip: true)
  .encoding(
    x: {field: "Palabras", type: "nominal", sort: "-y"},
    y: {field: "Frecuencia", type: "quantitative"}
  )

#<Vega::LiteChart:0x0000018ed78c4198 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "title"=>"Grafico 4 - Histograma de palabras en la Unión", "data"=>{:values=>[{:Palabras=>"abono", :Frecuencia=>1}, {:Palabras=>"para", :Frecuencia=>87}, {:Palabras=>"decia", :Frecuencia=>1}, {:Palabras=>"despues", :Frecuencia=>11}, {:Palabras=>"preguntan", :Frecuencia=>1}, {:Palabras=>"porque", :Frecuencia=>16}, {:Palabras=>"pocos", :Frecuencia=>3}, {:Palabras=>"pacientes", :Frecuencia=>2}, {:Palabras=>"tienen", :Frecuencia=>7}, {:Palabras=>"ojeriza", :Frecuencia=>1}, {:Palabras=>"medicos", :Frecuencia=>3}, {:Palabras=>"trabajan", :Frecuencia=>1}, {:Palabras=>"hospitales", :Frecuencia=>1}, {:Palabras=>"publicos", :Frecuencia=>1}, {:Palabras=>"todo", :Frecuencia=>36}, {:Palabras=>"enfermeras", :Frecuencia=>1}, {:Palabras=>"empaticos", :Frecuencia=>1}, {:Palabras=>"humanos", :Frecuencia=>3}, {:Palabras=>"merecen", :Frecuencia=>3}, {:Palabras=>"llamarse", :Frecuencia=>1}, {:Palabras=>"pero", :Frecuencia=>69}, {:Palabras=>"igual", :Frecuencia=>23}, {:Palabras=>"comia", :Frecuencia=>1}, {:Palabras=>"jugar", :Frecuencia=>19}, {:Palabras=>"pwro", :Frecuencia=>1}, {:Palabras=>"pareces", :Frecuencia=>1}, {:Palabras=>"patetica", :Frecuencia=>1}, {:Palabras=>"inutil", :Frecuencia=>1}, {:Palabras=>"osea", :Frecuencia=>1}, {:Palabras=>"hubieras", :Frecuencia=>1}, {:Palabras=>"visto", :Frecuencia=>9}, {:Palabras=>"siempre", :Frecuencia=>15}, {:Palabras=>"visitendo", :Frecuencia=>1}, {:Palabras=>"negro", :Frecuencia=>1}, {:Palabras=>"estando", :Frecuencia=>1}, {:Palabras=>"encerrado", :Frecuencia=>1}, {:Palabras=>"cuarto", :Frecuencia=>1}, {:Palabras=>"cuando", :Frecuencia=>36}, {:Palabras=>"descompuso", :Frecuencia=>1}, {:Palabras=>"escapar", :Frecuencia=>1}, {:Palabras=>"clases", :Frecuencia=>1}, {:Palabras=>"evitar", :Frecuencia=>1}, {:Palabras=>"bullyado", :Frecuencia=>1}, {:Palabras=>"life", :Frecuencia=>2}, {:Palabras=>"cada", :Frecuencia=>13}, {:Palabras=>"ando", :Frecuencia=>2}, {:Palabras=>"flex", :Frecuencia=>1}, {:Palabras=>"engordando", :Frecuencia=>1}, {:Palabras=>"racks", :Frecuencia=>2}, {:Palabras=>"bitch", :Frecuencia=>1}, {:Palabras=>"italia", :Frecuencia=>1}, {:Palabras=>"marca", :Frecuencia=>3}, {:Palabras=>"intentos", :Frecuencia=>1}, {:Palabras=>"increible", :Frecuencia=>2}, {:Palabras=>"cantidad", :Frecuencia=>2}, {:Palabras=>"oportunidades", :Frecuencia=>1}, {:Palabras=>"desperdicia", :Frecuencia=>1}, {:Palabras=>"volumen", :Frecuencia=>1}, {:Palabras=>"generacion", :Frecuencia=>1}, {:Palabras=>"juego", :Frecuencia=>5}, {:Palabras=>"necesitamos", :Frecuencia=>4}, {:Palabras=>"killer", :Frecuencia=>1}, {:Palabras=>"francotirador", :Frecuencia=>1}, {:Palabras=>"tipo", :Frecuencia=>1}, {:Palabras=>"escopeta", :Frecuencia=>1}, {:Palabras=>"rubio", :Frecuencia=>1}, {:Palabras=>"morocho", :Frecuencia=>1}, {:Palabras=>"zuluaga", :Frecuencia=>1}, {:Palabras=>"hijueputa", :Frecuencia=>1}, {:Palabras=>"depende", :Frecuencia=>1}, {:Palabras=>"salis", :Frecuencia=>1}, {:Palabras=>"barsito", :Frecuencia=>1}, {:Palabras=>"jean", :Frecuencia=>1}, {:Palabras=>"venis", :Frecuencia=>1}, {:Palabras=>"casa", :Frecuencia=>3}, {:Palabras=>"pongas", :Frecuencia=>4}, {:Palabras=>"nada", :Frecuencia=>34}, {:Palabras=>"esta", :Frecuencia=>56}, {:Palabras=>"noche", :Frecuencia=>5}, {:Palabras=>"saldre", :Frecuencia=>1}, {:Palabras=>"janguear", :Frecuencia=>1}, {:Palabras=>"tengo", :Frecuencia=>9}, {:Palabras=>"miedo", :Frecuencia=>6}, {:Palabras=>"verdad", :Frecuencia=>5}, {:Palabras=>"gusta", :Frecuencia=>13}, {:Palabras=>"mucho", :Frecuencia=>13}, {:Palabras=>"regalen", :Frecuencia=>1}, {:Palabras=>"flores", :Frecuencia=>1}, {:Palabras=>"mama", :Frecuencia=>4}, {:Palabras=>"descubra", :Frecuencia=>1}, {:Palabras=>"recien", :Frecuencia=>5}, {:Palabras=>"empezar", :Frecuencia=>2}, {:Palabras=>"asustar", :Frecuencia=>1}, {:Palabras=>"anda

### Número de tweets por fecha del conjunto 1

In [73]:
lista_por_fecha_1 = []
allTweets[:date].tally.each do |llave, valor|
  lista_por_fecha_1.push({date: llave, Frecuencia: valor})
end
puts "ok"

ok


#### Gráfico de tweets por fecha

In [74]:
Vega.lite
  .title("Grafico 5 - Histograma de Tweets por fecha del Conjunto 1")
  .data(lista_por_fecha_1)
  .transform([
      {
        "window": [{
          "op": "rank",
          "as": "rank"
        }],
        "sort": [{ "field": "Frecuencia", "order": "descending" }]
      }, {
        "filter": "datum.rank <= 20"
      }
    ])
  .mark(type: "bar", tooltip: true)
  .encoding(
    x: {field: "date", type: "nominal", sort: "-y"},
    y: {field: "Frecuencia", type: "quantitative"}
  )

#<Vega::LiteChart:0x0000018ed7cf0fa0 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "title"=>"Grafico 5 - Histograma de Tweets por fecha del Conjunto 1", "data"=>{:values=>[{:date=>"2021-11-06", :Frecuencia=>209}]}, "transform"=>[{:window=>[{:op=>"rank", :as=>"rank"}], :sort=>[{:field=>"Frecuencia", :order=>"descending"}]}, {:filter=>"datum.rank <= 20"}], "mark"=>{:type=>"bar", :tooltip=>true}, "encoding"=>{:x=>{:field=>"date", :type=>"nominal", :sort=>"-y"}, :y=>{:field=>"Frecuencia", :type=>"quantitative"}}}>

### Número de tweets por fecha del conjunto 2

In [75]:
lista_por_fecha_2 = []
tweets[:date].tally.each do |llave, valor|
  lista_por_fecha_2.push({date: llave, Frecuencia: valor})
end
puts "ok"

ok


In [85]:
Vega.lite
  .title("Grafico 5.2 - Histograma de Tweets por fecha del Conjunto 2")
  .data(lista_por_fecha_2)
  .transform([
      {
        "window": [{
          "op": "rank",
          "as": "rank"
        }],
        "sort": [{ "field": "Frecuencia", "order": "descending" }]
      }, {
        "filter": "datum.rank <= 20"
      }
    ])
  .mark(type: "bar", tooltip: true)
  .encoding(
    x: {field: "date", type: "nominal", sort: "-y"},
    y: {field: "Frecuencia", type: "quantitative"}
  )

#<Vega::LiteChart:0x00000295dd757158 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "title"=>"Grafico 5.2 - Histograma de Tweets por fecha del Conjunto 2", "data"=>{:values=>[{:date=>"2021-11-06", :Frecuencia=>25}, {:date=>"2021-11-05", :Frecuencia=>54}, {:date=>"2021-11-04", :Frecuencia=>39}, {:date=>"2021-11-03", :Frecuencia=>1}]}, "transform"=>[{:window=>[{:op=>"rank", :as=>"rank"}], :sort=>[{:field=>"Frecuencia", :order=>"descending"}]}, {:filter=>"datum.rank <= 20"}], "mark"=>{:type=>"bar", :tooltip=>true}, "encoding"=>{:x=>{:field=>"date", :type=>"nominal", :sort=>"-y"}, :y=>{:field=>"Frecuencia", :type=>"quantitative"}}}>

### Personas que twittearon por fecha

#### Para el conjunto 1

#### Agrupamos los username por fecha

In [76]:
grupo_1 = allTweets.group([:name, :date]).count
usuarios_fecha_1 = []
grupo_1[:date].tally.each do |llave, valor|
  usuarios_fecha_1.push({date: llave, Frecuencia: valor})
end

puts "ok"

ok


#### Gráfico

In [77]:
Vega.lite
  .title("Grafico 6.1 - Histograma de Usuarios por fecha del conjunto 1")
  .data(usuarios_fecha_1)
  .transform([
      {
        "window": [{
          "op": "rank",
          "as": "rank"
        }],
        "sort": [{ "field": "Frecuencia", "order": "descending" }]
      }, {
        "filter": "datum.rank <= 20"
      }
    ])
  .mark(type: "bar", tooltip: true)
  .encoding(
    x: {field: "date", type: "nominal", sort: "-y"},
    y: {field: "Frecuencia", type: "quantitative"}
  )

#<Vega::LiteChart:0x0000018ed8583b40 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "title"=>"Grafico 6.1 - Histograma de Usuarios por fecha del conjunto 1", "data"=>{:values=>[{:date=>"2021-11-06", :Frecuencia=>206}]}, "transform"=>[{:window=>[{:op=>"rank", :as=>"rank"}], :sort=>[{:field=>"Frecuencia", :order=>"descending"}]}, {:filter=>"datum.rank <= 20"}], "mark"=>{:type=>"bar", :tooltip=>true}, "encoding"=>{:x=>{:field=>"date", :type=>"nominal", :sort=>"-y"}, :y=>{:field=>"Frecuencia", :type=>"quantitative"}}}>

#### Para el conjunto 2

#### Agrupamos los username por fecha

In [78]:
grupo_2 = tweets.group([:name, :date]).count
usuarios_fecha_2 = []
grupo_2[:date].tally.each do |llave, valor|
  usuarios_fecha_2.push({date: llave, Frecuencia: valor})
end

puts "ok"

ok


#### Gráfico

In [79]:
Vega.lite
  .title("Grafico 6.2 - Histograma de Usuarios por fecha del conjunto 2")
  .data(usuarios_fecha_2)
  .transform([
      {
        "window": [{
          "op": "rank",
          "as": "rank"
        }],
        "sort": [{ "field": "Frecuencia", "order": "descending" }]
      }, {
        "filter": "datum.rank <= 20"
      }
    ])
  .mark(type: "bar", tooltip: true)
  .encoding(
    x: {field: "date", type: "nominal", sort: "-y"},
    y: {field: "Frecuencia", type: "quantitative"}
  )

#<Vega::LiteChart:0x0000018ed874f4b0 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "title"=>"Grafico 6.2 - Histograma de Usuarios por fecha del conjunto 2", "data"=>{:values=>[{:date=>"2021-11-06", :Frecuencia=>21}, {:date=>"2021-11-05", :Frecuencia=>51}, {:date=>"2021-11-04", :Frecuencia=>39}, {:date=>"2021-11-03", :Frecuencia=>36}, {:date=>"2021-11-02", :Frecuencia=>32}, {:date=>"2021-11-01", :Frecuencia=>27}]}, "transform"=>[{:window=>[{:op=>"rank", :as=>"rank"}], :sort=>[{:field=>"Frecuencia", :order=>"descending"}]}, {:filter=>"datum.rank <= 20"}], "mark"=>{:type=>"bar", :tooltip=>true}, "encoding"=>{:x=>{:field=>"date", :type=>"nominal", :sort=>"-y"}, :y=>{:field=>"Frecuencia", :type=>"quantitative"}}}>

### Número de tweets por usuario

#### En el conjunto 1

In [80]:
tweets_usuario = []
allTweets[:name].tally.each do |llave, valor|
  tweets_usuario.push({name: llave, Frecuencia: valor})
end
puts "ok"

ok


#### Gráfico

In [81]:
Vega.lite
  .title("Grafico 7.1 - Histograma de Tweets por usuario del conjunto 1")
  .data(tweets_usuario)
  .transform([
      {
        "window": [{
          "op": "rank",
          "as": "rank"
        }],
        "sort": [{ "field": "Frecuencia", "order": "descending" }]
      }, {
        "filter": "datum.rank <= 20"
      }
    ])
  .mark(type: "bar", tooltip: true)
  .encoding(
    x: {field: "name", type: "nominal", sort: "-y"},
    y: {field: "Frecuencia", type: "quantitative"}
  )

#<Vega::LiteChart:0x0000018ed87acd40 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "title"=>"Grafico 7.1 - Histograma de Tweets por usuario del conjunto 1", "data"=>{:values=>[{:name=>"fatigakinnie", :Frecuencia=>1}, {:name=>"eugeniaperret", :Frecuencia=>1}, {:name=>"SadLouie97", :Frecuencia=>1}, {:name=>"gikaben", :Frecuencia=>1}, {:name=>"Just_iFrann", :Frecuencia=>1}, {:name=>"matoelreproche", :Frecuencia=>1}, {:name=>"jaimebarriosre1", :Frecuencia=>1}, {:name=>"Luujuares", :Frecuencia=>1}, {:name=>"julcepla", :Frecuencia=>1}, {:name=>"Conniedd_", :Frecuencia=>1}, {:name=>"Gooppe", :Frecuencia=>1}, {:name=>"Johalisssss", :Frecuencia=>1}, {:name=>"redeescorpiana", :Frecuencia=>1}, {:name=>"vigiadevillegas", :Frecuencia=>1}, {:name=>"VSensuales", :Frecuencia=>1}, {:name=>"kren_souza", :Frecuencia=>1}, {:name=>"NeruzRap", :Frecuencia=>1}, {:name=>"i_electronics22", :Frecuencia=>1}, {:name=>"DMUruguay3", :Frecuencia=>1}, {:name=>"FalklandsWars", :Frecuencia=>1}, {:name=>"joselara27", :Frecuencia=>1}, {:name=>"Kiarataku", :Frecuencia=>1}, {:name=>"vallesadrian", :Frecuencia=>1}, {:name=>"Katyblxck_cat", :Frecuencia=>1}, {:name=>"arnaiz_pili", :Frecuencia=>1}, {:name=>"OscardarioRend1", :Frecuencia=>1}, {:name=>"ivanarmesto66", :Frecuencia=>1}, {:name=>"Claros_W0", :Frecuencia=>3}, {:name=>"dragonaweasley", :Frecuencia=>1}, {:name=>"criisttinaacg", :Frecuencia=>1}, {:name=>"domenec_ortiz", :Frecuencia=>1}, {:name=>"Noralobbo", :Frecuencia=>1}, {:name=>"kyxgre", :Frecuencia=>1}, {:name=>"Vickyceb051", :Frecuencia=>1}, {:name=>"StevenLozano_93", :Frecuencia=>1}, {:name=>"fleurstrap", :Frecuencia=>1}, {:name=>"Figoleon", :Frecuencia=>1}, {:name=>"eerror1101", :Frecuencia=>1}, {:name=>"prefratterista", :Frecuencia=>1}, {:name=>"AlianzaCivicaNi", :Frecuencia=>1}, {:name=>"SoyEros_", :Frecuencia=>1}, {:name=>"CesarAngelo17", :Frecuencia=>1}, {:name=>"Pvillamuro", :Frecuencia=>1}, {:name=>"luanixx_e", :Frecuencia=>1}, {:name=>"Ro_Almagro", :Frecuencia=>1}, {:name=>"boahes", :Frecuencia=>1}, {:name=>"iace16", :Frecuencia=>1}, {:name=>"KotegawaTyran", :Frecuencia=>1}, {:name=>"axere_yo", :Frecuencia=>1}, {:name=>"Alizon05176275", :Frecuencia=>1}, {:name=>"ADELyzed36", :Frecuencia=>1}, {:name=>"_Otsuwa_", :Frecuencia=>1}, {:name=>"oscar_or_lgs", :Frecuencia=>1}, {:name=>"Ernesto3108Yt", :Frecuencia=>1}, {:name=>"tundetecla", :Frecuencia=>1}, {:name=>"rcedano", :Frecuencia=>1}, {:name=>"Lord_SxTwo", :Frecuencia=>1}, {:name=>"Forest_Rechazo", :Frecuencia=>1}, {:name=>"Srgiortega", :Frecuencia=>1}, {:name=>"_gidliz", :Frecuencia=>1}, {:name=>"lizmenfor", :Frecuencia=>1}, {:name=>"woobeari", :Frecuencia=>1}, {:name=>"lupitasosa", :Frecuencia=>1}, {:name=>"Raul_OrtizS", :Frecuencia=>1}, {:name=>"santhros", :Frecuencia=>1}, {:name=>"sanfilosofo", :Frecuencia=>1}, {:name=>"Zapata78145121", :Frecuencia=>1}, {:name=>"AliSierra_Lcdp", :Frecuencia=>1}, {:name=>"Leocollino2", :Frecuencia=>1}, {:name=>"PasenContext0", :Frecuencia=>1}, {:name=>"Jodies_Wife", :Frecuencia=>1}, {:name=>"tetsichan", :Frecuencia=>1}, {:name=>"alexanderfrias2", :Frecuencia=>1}, {:name=>"Callmeyisus_", :Frecuencia=>1}, {:name=>"rrolahorca", :Frecuencia=>1}, {:name=>"ConradoMora1", :Frecuencia=>1}, {:name=>"NoSoyLaTostada", :Frecuencia=>1}, {:name=>"gourflores", :Frecuencia=>1}, {:name=>"_BlackPhillip_6", :Frecuencia=>1}, {:name=>"soyherflor", :Frecuencia=>1}, {:name=>"myneai", :Frecuencia=>1}, {:name=>"eganez_e", :Frecuencia=>1}, {:name=>"Asha_Yozako", :Frecuencia=>1}, {:name=>"Tortugu46591777", :Frecuencia=>1}, {:name=>"Boquita012", :Frecuencia=>1}, {:name=>"SophieScholl17", :Frecuencia=>1}, {:name=>"mamitornasol", :Frecuencia=>1}, {:name=>"Jfgoitiac", :Frecuencia=>1}, {:name=>"AlanStarobinsky", :Frecuencia=>1}, {:name=>"abaquerizog", :Frecuencia=>1}, {:name=>"tzunaranjita", :Frecuencia=>1}, {:name=>"1Dolar1000Pesos", :Frecuencia=>1}, {:name=>"cutiese

#### En el conjunto 2

In [82]:
allTweets_usuario = []
tweets[:name].tally.each do |llave, valor|
  allTweets_usuario.push({name: llave, Frecuencia: valor})
end
puts "ok"

ok


#### Gráfico

In [88]:
Vega.lite
  .title("Grafico 7.2 - Histograma de Tweets por usuario del conjunto 2")
  .data(allTweets_usuario)
  .transform([
      {
        "window": [{
          "op": "rank",
          "as": "rank"
        }],
        "sort": [{ "field": "Frecuencia", "order": "descending" }]
      }, {
        "filter": "datum.rank <= 5"
      }
    ])
  .mark(type: "bar", tooltip: true)
  .encoding(
    x: {field: "name", type: "nominal", sort: "-y"},
    y: {field: "Frecuencia", type: "quantitative"}
  )

#<Vega::LiteChart:0x0000018ed8a65de8 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "title"=>"Grafico 7.2 - Histograma de Tweets por usuario del conjunto 2", "data"=>{:values=>[{:name=>"LepereIorio", :Frecuencia=>2}, {:name=>"Antonio47256118", :Frecuencia=>1}, {:name=>"MAURICIO1905LM", :Frecuencia=>2}, {:name=>"imoralesglez", :Frecuencia=>1}, {:name=>"SkoOrp", :Frecuencia=>1}, {:name=>"borjacouce", :Frecuencia=>1}, {:name=>"melixplaz", :Frecuencia=>2}, {:name=>"Tobioysu14", :Frecuencia=>1}, {:name=>"ALLIN45", :Frecuencia=>1}, {:name=>"josubilbok", :Frecuencia=>1}, {:name=>"CarlosXArguello", :Frecuencia=>1}, {:name=>"OzunaVictor78", :Frecuencia=>1}, {:name=>"cris97diaz", :Frecuencia=>1}, {:name=>"jfsoltero23", :Frecuencia=>2}, {:name=>"OscarLpez7", :Frecuencia=>3}, {:name=>"RaulOrregoC", :Frecuencia=>1}, {:name=>"MarcialOrueS1", :Frecuencia=>1}, {:name=>"Gus_fernan", :Frecuencia=>1}, {:name=>"maldiniccp", :Frecuencia=>1}, {:name=>"TicTacEfimero1", :Frecuencia=>1}, {:name=>"MarcosSJacquet", :Frecuencia=>1}, {:name=>"miguelangel_PRY", :Frecuencia=>1}, {:name=>"PabloArg9", :Frecuencia=>1}, {:name=>"asismartin", :Frecuencia=>1}, {:name=>"yarontecuenta", :Frecuencia=>1}, {:name=>"Pelota_movi1", :Frecuencia=>2}, {:name=>"LopezIbieta", :Frecuencia=>1}, {:name=>"Ren794", :Frecuencia=>1}, {:name=>"siberianostro", :Frecuencia=>1}, {:name=>"crimalopz", :Frecuencia=>2}, {:name=>"Fanalepra", :Frecuencia=>1}, {:name=>"MolinasEmery", :Frecuencia=>1}, {:name=>"adrianbasu10", :Frecuencia=>1}, {:name=>"Frutos_07", :Frecuencia=>1}, {:name=>"JulioPe76251750", :Frecuencia=>1}, {:name=>"24HDeportesTVN", :Frecuencia=>1}, {:name=>"elprosor123", :Frecuencia=>1}, {:name=>"danyarevalos", :Frecuencia=>1}, {:name=>"luchoalvaren", :Frecuencia=>1}, {:name=>"LomaquisLuis", :Frecuencia=>1}, {:name=>"AngusDuarte", :Frecuencia=>1}, {:name=>"SL_GUMA", :Frecuencia=>1}, {:name=>"NicoOk04", :Frecuencia=>2}, {:name=>"pablomtf", :Frecuencia=>1}, {:name=>"CuevasRodrigo91", :Frecuencia=>1}, {:name=>"AnibalR1373", :Frecuencia=>1}, {:name=>"MinutoDecano", :Frecuencia=>1}, {:name=>"anoemiS22", :Frecuencia=>1}, {:name=>"jorgesosabogado", :Frecuencia=>1}, {:name=>"AlexisYudizKiri", :Frecuencia=>1}, {:name=>"tincho_pe", :Frecuencia=>1}, {:name=>"Pewmafe16", :Frecuencia=>1}, {:name=>"daniel_francx", :Frecuencia=>1}, {:name=>"Jhonfer24set", :Frecuencia=>1}, {:name=>"ivanrafael2002", :Frecuencia=>1}, {:name=>"DecirMejor", :Frecuencia=>1}, {:name=>"josenegrosfc6", :Frecuencia=>1}, {:name=>"luisaraujo1957", :Frecuencia=>1}, {:name=>"ACJ_abogado", :Frecuencia=>1}, {:name=>"MMCOSFC1", :Frecuencia=>7}, {:name=>"Jo_Ka", :Frecuencia=>1}, {:name=>"Johnhermosilla1", :Frecuencia=>1}, {:name=>"FutbolCatracho", :Frecuencia=>1}, {:name=>"mauroGODselli9", :Frecuencia=>1}, {:name=>"ricardoobando", :Frecuencia=>1}, {:name=>"ezemercadito4", :Frecuencia=>1}, {:name=>"SerranaNovelera", :Frecuencia=>1}, {:name=>"Levinsito", :Frecuencia=>1}, {:name=>"lucho_maradei", :Frecuencia=>1}, {:name=>"blazer161175", :Frecuencia=>1}, {:name=>"Avila84Marcos", :Frecuencia=>1}, {:name=>"Mateelujan_", :Frecuencia=>1}, {:name=>"paomjara", :Frecuencia=>1}, {:name=>"AlejandroNimo1", :Frecuencia=>1}, {:name=>"elciudadanoweb", :Frecuencia=>1}, {:name=>"ZanabriaDeZurda", :Frecuencia=>1}, {:name=>"hunagaro", :Frecuencia=>1}, {:name=>"SantacrVera", :Frecuencia=>1}, {:name=>"Jorge_Canob", :Frecuencia=>1}, {:name=>"torero5x0", :Frecuencia=>1}, {:name=>"UnHinchaLeproso", :Frecuencia=>1}, {:name=>"Xavierthv", :Frecuencia=>1}, {:name=>"carlosnietooo", :Frecuencia=>1}, {:name=>"FerriolJorge", :Frecuencia=>1}, {:name=>"jlc898", :Frecuencia=>1}, {:name=>"Phenompy", :Frecuencia=>1}, {:name=>"luiscorderoBSC", :Frecuencia=>1}, {:name=>"JorgeDjoubaili", :Frecuencia=>1}, {:name=>"gabocaraba", :Frecuencia=>1}, {:name=>"chino_canob", :Frecuencia=>1}, {:name=>"Marioxerecista", :Frecuencia=>1}, {:name=>"MauricioOrt

### Diferencias entre conjuntos

#### Conjunto1 - Conjunto2

In [84]:
lista1 = []
# guardamos en una lista todas las palabras del conjunto 1
conjunto_1[:Palabras].each do |valor|
  lista1.push(valor)
end

lista2 = []
# guardamos en una lista todas las palabras del conjunto 2
conjunto_2[:Palabras].each do |valor|
  lista2.push(valor)
end

# diferencia Conjunto1 - Conjunto2
r = lista1-lista2
# diferencia Conjunto2 - Conjunto1
s = lista2-lista1

# hashes que contendran los numeros de repeticiones de cada palabra
# 1 - 2
h = Hash.new(0)
# 2 - 1
i = Hash.new(0)

# buscamos las repeticiones
r.each { |v| h.store(v, h[v]+1) }

s.each { |m| i.store(m, i[m]+1) }

uno_no_dos = []
dos_no_uno = []

# creamos una lista de hashes para el grafico de la diferencia Conjunto1 - Conjunto2
h.each do |llave, valor|
  uno_no_dos.push({Palabras: llave, Frecuencia: valor})
end

# creamos una lista de hashes para el grafico de la diferencia Conjunto2 - Conjunto1
i.each do |llave, valor|
  dos_no_uno.push({Palabras: llave, Frecuencia: valor})
end



{}

#### Conjunto2 - Conjunto1

### Gráfico de las diferencias

#### Diferencia Conjunto1 - Conjunto2

In [85]:
Vega.lite
  .title("Grafico 8.1 - Histograma de palabras de Conjunto1 - Conjunto2")
  .data(uno_no_dos)
  .transform([
      {
        "window": [{
          "op": "rank",
          "as": "rank"
        }],
        "sort": [{ "field": "Frecuencia", "order": "descending" }]
      }, {
        "filter": "datum.rank <= 20"
      }
    ])
  .mark(type: "bar", tooltip: true)
  .encoding(
    x: {field: "Palabras", type: "nominal", sort: "-y"},
    y: {field: "Frecuencia", type: "quantitative"}
  )

#<Vega::LiteChart:0x0000018ed88dcad0 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "title"=>"Grafico 8.1 - Histograma de palabras de Conjunto1 - Conjunto2", "data"=>{:values=>[{:Palabras=>"abono", :Frecuencia=>1}, {:Palabras=>"decia", :Frecuencia=>1}, {:Palabras=>"preguntan", :Frecuencia=>1}, {:Palabras=>"pacientes", :Frecuencia=>2}, {:Palabras=>"ojeriza", :Frecuencia=>1}, {:Palabras=>"medicos", :Frecuencia=>3}, {:Palabras=>"trabajan", :Frecuencia=>1}, {:Palabras=>"hospitales", :Frecuencia=>1}, {:Palabras=>"publicos", :Frecuencia=>1}, {:Palabras=>"enfermeras", :Frecuencia=>1}, {:Palabras=>"empaticos", :Frecuencia=>1}, {:Palabras=>"humanos", :Frecuencia=>3}, {:Palabras=>"llamarse", :Frecuencia=>1}, {:Palabras=>"comia", :Frecuencia=>1}, {:Palabras=>"pwro", :Frecuencia=>1}, {:Palabras=>"pareces", :Frecuencia=>1}, {:Palabras=>"patetica", :Frecuencia=>1}, {:Palabras=>"inutil", :Frecuencia=>1}, {:Palabras=>"osea", :Frecuencia=>1}, {:Palabras=>"hubieras", :Frecuencia=>1}, {:Palabras=>"visitendo", :Frecuencia=>1}, {:Palabras=>"negro", :Frecuencia=>1}, {:Palabras=>"estando", :Frecuencia=>1}, {:Palabras=>"encerrado", :Frecuencia=>1}, {:Palabras=>"cuarto", :Frecuencia=>1}, {:Palabras=>"descompuso", :Frecuencia=>1}, {:Palabras=>"escapar", :Frecuencia=>1}, {:Palabras=>"clases", :Frecuencia=>1}, {:Palabras=>"evitar", :Frecuencia=>1}, {:Palabras=>"bullyado", :Frecuencia=>1}, {:Palabras=>"life", :Frecuencia=>2}, {:Palabras=>"ando", :Frecuencia=>2}, {:Palabras=>"flex", :Frecuencia=>1}, {:Palabras=>"engordando", :Frecuencia=>1}, {:Palabras=>"racks", :Frecuencia=>2}, {:Palabras=>"bitch", :Frecuencia=>1}, {:Palabras=>"italia", :Frecuencia=>1}, {:Palabras=>"intentos", :Frecuencia=>1}, {:Palabras=>"increible", :Frecuencia=>2}, {:Palabras=>"cantidad", :Frecuencia=>2}, {:Palabras=>"oportunidades", :Frecuencia=>1}, {:Palabras=>"desperdicia", :Frecuencia=>1}, {:Palabras=>"volumen", :Frecuencia=>1}, {:Palabras=>"generacion", :Frecuencia=>1}, {:Palabras=>"killer", :Frecuencia=>1}, {:Palabras=>"francotirador", :Frecuencia=>1}, {:Palabras=>"tipo", :Frecuencia=>1}, {:Palabras=>"escopeta", :Frecuencia=>1}, {:Palabras=>"rubio", :Frecuencia=>1}, {:Palabras=>"morocho", :Frecuencia=>1}, {:Palabras=>"zuluaga", :Frecuencia=>1}, {:Palabras=>"hijueputa", :Frecuencia=>1}, {:Palabras=>"depende", :Frecuencia=>1}, {:Palabras=>"salis", :Frecuencia=>1}, {:Palabras=>"barsito", :Frecuencia=>1}, {:Palabras=>"jean", :Frecuencia=>1}, {:Palabras=>"venis", :Frecuencia=>1}, {:Palabras=>"casa", :Frecuencia=>3}, {:Palabras=>"noche", :Frecuencia=>5}, {:Palabras=>"saldre", :Frecuencia=>1}, {:Palabras=>"janguear", :Frecuencia=>1}, {:Palabras=>"miedo", :Frecuencia=>6}, {:Palabras=>"regalen", :Frecuencia=>1}, {:Palabras=>"flores", :Frecuencia=>1}, {:Palabras=>"mama", :Frecuencia=>4}, {:Palabras=>"descubra", :Frecuencia=>1}, {:Palabras=>"empezar", :Frecuencia=>2}, {:Palabras=>"asustar", :Frecuencia=>1}, {:Palabras=>"soho", :Frecuencia=>1}, {:Palabras=>"hiper", :Frecuencia=>1}, {:Palabras=>"clasica", :Frecuencia=>1}, {:Palabras=>"antro", :Frecuencia=>1}, {:Palabras=>"bens", :Frecuencia=>1}, {:Palabras=>"cuadra", :Frecuencia=>1}, {:Palabras=>"washington", :Frecuencia=>1}, {:Palabras=>"park", :Frecuencia=>1}, {:Palabras=>"greenwich", :Frecuencia=>1}, {:Palabras=>"pensados", :Frecuencia=>1}, {:Palabras=>"esas", :Frecuencia=>2}, {:Palabras=>"maletas", :Frecuencia=>1}, {:Palabras=>"cartillas", :Frecuencia=>1}, {:Palabras=>"morales", :Frecuencia=>1}, {:Palabras=>"iban", :Frecuencia=>1}, {:Palabras=>"repartir", :Frecuencia=>1}, {:Palabras=>"favor", :Frecuencia=>3}, {:Palabras=>"corregir", :Frecuencia=>1}, {:Palabras=>"gastaron", :Frecuencia=>1}, {:Palabras=>"pesos", :Frecuencia=>2}, {:Palabras=>"tendremos", :Frecuencia=>3}, {:Palabras=>"cartilla", :Frecuencia=>1}, {:Palabras=>"felicidad", :Frecuencia=>2}, {:Palabras=>"caminar", :Frecuencia=>1}, {:Palabras=>"chacao", :Fre

#### Diferencia Conjunto2 - Conjunto1

In [86]:
Vega.lite
  .title("Grafico 8.2 - Histograma de palabras de Conjunto2 - Conjunto1")
  .data(dos_no_uno)
  .transform([
      {
        "window": [{
          "op": "rank",
          "as": "rank"
        }],
        "sort": [{ "field": "Frecuencia", "order": "descending" }]
      }, {
        "filter": "datum.rank <= 20"
      }
    ])
  .mark(type: "bar", tooltip: true)
  .encoding(
    x: {field: "Palabras", type: "nominal", sort: "-y"},
    y: {field: "Frecuencia", type: "quantitative"}
  )

#<Vega::LiteChart:0x0000018ed8a3e810 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "title"=>"Grafico 8.2 - Histograma de palabras de Conjunto2 - Conjunto1", "data"=>{:values=>[]}, "transform"=>[{:window=>[{:op=>"rank", :as=>"rank"}], :sort=>[{:field=>"Frecuencia", :order=>"descending"}]}, {:filter=>"datum.rank <= 20"}], "mark"=>{:type=>"bar", :tooltip=>true}, "encoding"=>{:x=>{:field=>"Palabras", :type=>"nominal", :sort=>"-y"}, :y=>{:field=>"Frecuencia", :type=>"quantitative"}}}>

In [252]:
a = ["a", "b", "c"]
b = ["b", "a", "f"]

b-a

["f"]